# Install Detectron2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/gdrive/MyDrive/keypoints_detection/keypoints_dataset/annotations_train.json", "/content/gdrive/MyDrive/keypoints_detection/keypoints_dataset/train")
register_coco_instances("my_dataset_val", {}, "/content/gdrive/MyDrive/keypoints_detection/keypoints_dataset/annotations_val.json", "/content/gdrive/MyDrive/keypoints_detection/keypoints_dataset/val")
register_coco_instances("my_dataset_test", {}, "/content/gdrive/MyDrive/keypoints_detection/keypoints_dataset/annotations_test.json", "/content/gdrive/MyDrive/keypoints_detection/keypoints_dataset/test")

In [ ]:
MetadataCatalog.get("my_dataset_train").keypoint_names = ["head", "tail"]
MetadataCatalog.get("my_dataset_train").keypoint_flip_map = []
MetadataCatalog.get("my_dataset_test").keypoint_names = ["head", "tail"]
MetadataCatalog.get("my_dataset_test").keypoint_flip_map = []
val = MetadataCatalog.get("my_dataset_train")
print(val)

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.RETINANET.NUM_CLASSES = 3
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 2
cfg.OUTPUT_DIR = "/content/gdrive/MyDrive/output"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
from detectron2.utils.visualizer import ColorMode
import matplotlib.pyplot as plt
import json

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml"))
cfg.OUTPUT_DIR = "/content/gdrive/MyDrive/output"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the trained model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8   # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.MODEL.RETINANET.NUM_CLASSES = 3
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 2
cfg.MODEL.DEVICE = "cpu"
predictor = DefaultPredictor(cfg)

img = cv2.imread(f'/content/gdrive/MyDrive/img1.png', cv2.IMREAD_UNCHANGED)#

# fix transparency error with
trans_mask = img[:,:,3] == 0
img[trans_mask] = [255, 255, 255, 255]
img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

outputs = predictor(img)
print(len(outputs.get("instances").pred_boxes.tensor.numpy()))
print(outputs.get("instances").pred_boxes.tensor.numpy())
print(outputs.get("instances").pred_keypoints.cpu().numpy())

for kp in outputs.get("instances").pred_keypoints.cpu().numpy():
  cv2.circle(img, (kp[0][0],kp[0][1]), 4, (0,255,0), -1)
  cv2.circle(img, (kp[1][0],kp[1][1]), 4, (0,0,255), -1)
cv2_imshow(img)

v = Visualizer(img[:, :, ::-1],
                metadata=val,
                scale=2,
                instance_mode=ColorMode.IMAGE_BW
)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/gdrive/MyDrive/output"

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_test", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`